Ethics of AI Final Project

In [143]:
!pip install datasets

^C


In [49]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import  DecisionTreeClassifier ,plot_tree, export_text# Decision tree algorithm and plotting functions for the Decision tree
from matplotlib import pyplot as plt # plotting/graphing
import numpy as np
from itertools import product
from sklearn.utils import class_weight


In [50]:
# fetch dataset and apply processing file
dataset = load_dataset("mstz/speeddating",trust_remote_code=True)["train"]

# convert to df
df = pd.DataFrame(dataset)


# split into x and y sets
X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date']) # is match is our target variable, which is determined from the last 2 so we need to drop all
X = pd.get_dummies(X) #get dummies for X
y = df.is_match

X.columns


Index(['is_dater_male', 'dater_age', 'dated_age', 'age_difference',
       'are_same_race', 'same_race_importance_for_dater',
       'same_religion_importance_for_dater',
       'attractiveness_importance_for_dated', 'sincerity_importance_for_dated',
       'intelligence_importance_for_dated', 'humor_importance_for_dated',
       'ambition_importance_for_dated',
       'shared_interests_importance_for_dated',
       'attractiveness_score_of_dater_from_dated',
       'sincerity_score_of_dater_from_dated',
       'intelligence_score_of_dater_from_dated',
       'humor_score_of_dater_from_dated', 'ambition_score_of_dater_from_dated',
       'shared_interests_score_of_dater_from_dated',
       'attractiveness_importance_for_dater', 'sincerity_importance_for_dater',
       'intelligence_importance_for_dater', 'humor_importance_for_dater',
       'ambition_importance_for_dater',
       'shared_interests_importance_for_dater',
       'self_reported_attractiveness_of_dater',
       'self_repor

In [51]:
# split into training and tests, ALL RANDOM STATES SET TO 0
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .33,random_state=0)

In [52]:
positive_weight = 15  # You can adjust this weight as needed
negative_weight = 1  # You can adjust this weight as needed

# Create a dictionary to specify class weights
class_weights = {0: negative_weight, 1: positive_weight}

In [53]:
# Different models
# Random Forest

#Set estimators as 100, random state 0, samples of at least 2 in each leaf
clf = RandomForestClassifier(class_weight={0: class_weights[0], 1: class_weights[1]},n_estimators = 200, random_state=0,min_samples_leaf=6)
clf.fit(x_train, y_train) # fit model

# get the roc auc and pring
cross_val_accuracy_roc_auc = (cross_val_score(clf, x_train, y_train, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_accuracy_roc_auc

84.73032714412024

# 1 Pre Processing
Our dataset has a variety of features. We will want to simplify our dataset to have a few key features:
Age, age difference for the pairing (Sensitive)
Race, same race for the pairing (Sensitive)
expected_number_of_likes_of_dater_from_20_people (non sensitive)
Already met before (non sensitive)


In [7]:
features = ['dater_age','dated_age','age_difference','dater_race','dated_race','are_same_race','expected_number_of_likes_of_dater_from_20_people','already_met_before']

X = df[features].copy()
y = df.is_match.copy()

# We will want to see where the race is the same or not

print('Matches where pairings are the same race: ',len(X[X['are_same_race']]))
print('Matches where pairings are not the same race: ',len(X[-X['are_same_race']]))

Matches where pairings are the same race:  437
Matches where pairings are not the same race:  611


In [8]:
print('Matches where pairings are the same race and match: ',len(X[X['are_same_race'] & y==1]))
print('Matches where pairings are not the same race and match: ',len(X[-X['are_same_race'] & y==1]))

Matches where pairings are the same race and match:  84
Matches where pairings are not the same race and match:  102


# Split data and Examine Match Rates by Group

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state= 372)

print('% Same Race in train: ', len(X_train[X_train['are_same_race']])/len(X_train))
print('% Not Same Race in train: ', len(X_train[-X_train['are_same_race']])/len(X_train))

print('%  Same Race in test: ', len(X_test[X_test['are_same_race']])/len(X_test))
print('% Not Same Race test: ', len(X_test[-X_test['are_same_race']])/len(X_test))

#make a df where the couple is a match
train_is_match = X_train.join(y_train)
train_is_match = train_is_match[y_train==1]

print('Difference in Match Rate between Groups in Training Data: ',
    ( (len(train_is_match[train_is_match['are_same_race']]) / len(X_train[X_train['are_same_race']]))
     -
    ( (len(train_is_match[-train_is_match['are_same_race']])) / len(X_train[-X_train['are_same_race']])))

)

% Same Race in train:  0.4092769440654843
% Not Same Race in train:  0.5907230559345157
%  Same Race in test:  0.43492063492063493
% Not Same Race test:  0.5650793650793651
Difference in Match Rate between Groups in Training Data:  0.029622786759045422


Metrics by Race

In [11]:
results = x_test
y_hat = clf.predict(x_test)
results['predicted'] = y_hat
results['actual'] = y_test

In [10]:
results.head()

NameError: name 'results' is not defined

In [12]:
dater_groups = ["dater_race_'Asian/Pacific Islander/Asian-American'","dater_race_'Black/African American'","dater_race_'Latino/Hispanic American'", 'dater_race_caucasian','dater_race_other']
dated_groups = ["dated_race_'Asian/Pacific Islander/Asian-American'","dated_race_'Black/African American'","dated_race_'Latino/Hispanic American'", 'dated_race_caucasian','dated_race_other']

def compute_metrics(info, group):
  metric_df = info.loc[info[group] == 1]
  pred_pos = sum(metric_df['predicted'])
  act_pos = sum(metric_df['actual'])

  positive_rate = pred_pos/len(metric_df)
  accuracy = len(metric_df[metric_df['predicted'] == metric_df['actual']])/len(metric_df)
  tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
  fp = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 1)])
  fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
  tn = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 0)])
  tpr = tp/(tp + fn )
  fpr = fp/(fp + tn)
  precision = tp/(tp + fp)
  recall = tp/(tp + fn)

  row = [group, accuracy, precision, recall, positive_rate, tpr, fpr]

  return row

column_names = ['group', 'accuracy', 'precision', 'recall', 'positive_rate', 'tpr', 'fpr']
metrics_df = pd.DataFrame(columns=column_names)
for group in dater_groups:
  row = compute_metrics(results, group)
  metrics_df.loc[len(metrics_df)] = row

for group in dated_groups:
  row = compute_metrics(results, group)
  metrics_df.loc[len(metrics_df)] = row

metrics_df

ZeroDivisionError: division by zero

## 2 - Inprocess Bias Mitigation

In [ ]:
df = pd.DataFrame(dataset)

X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date']) # is match is our target variable, which is determined from the last 2 so we need to drop all
X = pd.get_dummies(X) #get dummies for X
y = df.is_match

# Same race data split into y and X
X_same = df[df['are_same_race'] == 1]
y_same = X_same.is_match
X_same = X_same.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date'])
X_same = pd.get_dummies(X_same) #get dummies for X

# Different race data split into y and x
X_diff = df[df['are_same_race'] == 0]
y_diff = X_diff.is_match
X_diff = X_diff.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date'])
X_diff = pd.get_dummies(X_diff) #get dummies for X

x_train_same, x_test_same, y_train_same, y_test_same = train_test_split(X_same, y_same, test_size = .33,random_state=0)
x_train_diff, x_test_diff, y_train_diff, y_test_diff = train_test_split(X_diff, y_diff, test_size = .33,random_state=0)

clf1 = RandomForestClassifier(n_estimators = 125, random_state=0, min_samples_leaf=2)
clf1.fit(x_train_same, y_train_same) # fit model

clf2 = RandomForestClassifier(n_estimators = 125, random_state=0, min_samples_leaf=2)
clf2.fit(x_train_diff, y_train_diff) # fit model

cross_val_dict = {}
# get the roc auc and pring
# Diff
cross_val_accuracy_roc_auc = (cross_val_score(clf1, x_train_same, y_train_same, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_dict['same'] = cross_val_accuracy_roc_auc

# Diff
cross_val_accuracy_roc_auc = (cross_val_score(clf1, x_train_diff, y_train_diff, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_dict['diff'] = cross_val_accuracy_roc_auc

print(cross_val_dict)


{'same': 82.29589371980677, 'diff': 81.3318054494525}


In [ ]:
# Same results
results_same = x_test_same
y_hat = clf1.predict(x_test_same)
results_same['predicted'] = y_hat
results_same['actual'] = y_test_same

# Diff results
results_diff = x_test_diff
y_hat = clf2.predict(x_test_diff)
results_diff['predicted'] = y_hat
results_diff['actual'] = y_test_diff

results = pd.concat([results_same, results_diff])

In [ ]:
results.head()

,is_dater_male,dater_age,dated_age,age_difference,are_same_race,same_race_importance_for_dater,same_religion_importance_for_dater,attractiveness_importance_for_dated,sincerity_importance_for_dated,intelligence_importance_for_dated,...,dater_race_'Latino/Hispanic American',dater_race_caucasian,dater_race_other,dated_race_'Asian/Pacific Islander/Asian-American',dated_race_'Black/African American',dated_race_'Latino/Hispanic American',dated_race_caucasian,dated_race_other,predicted,actual
601,False,34,24,10,True,5.0,7.0,10.0,20.0,35.0,...,False,True,False,False,False,False,True,False,0,0
694,False,30,26,4,True,3.0,5.0,18.0,18.0,18.0,...,False,True,False,False,False,False,True,False,0,0
764,False,29,26,3,True,1.0,2.0,18.0,18.0,18.0,...,False,True,False,False,False,False,True,False,0,0
453,True,28,23,5,True,2.0,2.0,10.0,20.0,20.0,...,False,True,False,False,False,False,True,False,0,0
149,False,27,30,3,True,2.0,3.0,20.0,20.0,25.0,...,False,True,False,False,False,False,True,False,0,0


In [ ]:
dater_groups = ["dater_race_'Asian/Pacific Islander/Asian-American'","dater_race_'Black/African American'","dater_race_'Latino/Hispanic American'", 'dater_race_caucasian','dater_race_other']
dated_groups = ["dated_race_'Asian/Pacific Islander/Asian-American'","dated_race_'Black/African American'","dated_race_'Latino/Hispanic American'", 'dated_race_caucasian','dated_race_other']

def safe_divide(n, d):
  try:
    return n/d
  except ZeroDivisionError:
    return np.nan


def compute_metrics(info, group):
  metric_df = info.loc[info[group] == 1]
  pred_pos = sum(metric_df['predicted'])
  act_pos = sum(metric_df['actual'])

  positive_rate = pred_pos/len(metric_df)
  accuracy = len(metric_df[metric_df['predicted'] == metric_df['actual']])/len(metric_df)
  tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
  fp = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 1)])
  fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
  tn = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 0)])
  tpr = safe_divide(tp, (tp + fn))
  fpr = safe_divide(fp, (fp + tn))
  precision = safe_divide(tp, (tp + fp))
  recall = safe_divide(tp, (tp + fn))

  row = [group, accuracy, precision, recall, positive_rate, tpr, fpr]

  return row






In [15]:
X.value_counts(subset='dated_race')

dated_race
caucasian                                  641
'Asian/Pacific Islander/Asian-American'    196
'Latino/Hispanic American'                  77
other                                       70
'Black/African American'                    64
dtype: int64

In [ ]:
column_names = ['group', 'accuracy', 'precision', 'recall', 'positive_rate', 'tpr', 'fpr']
metrics_df2 = pd.DataFrame(columns=column_names)
for group in dater_groups:
  row = compute_metrics(results, group)
  metrics_df2.loc[len(metrics_df2)] = row

for group in dated_groups:
  row = compute_metrics(results, group)
  metrics_df2.loc[len(metrics_df2)] = row

metrics_df2

,group,accuracy,precision,recall,positive_rate,tpr,fpr
0,dater_race_'Asian/Pacific Islander/Asian-Ameri...,0.897436,0.500000,0.125000,0.025641,0.125000,0.014286
1,dater_race_'Black/African American',0.888889,1.000000,0.666667,0.222222,0.666667,0.000000
2,dater_race_'Latino/Hispanic American',0.870968,NaN,0.000000,0.000000,0.000000,0.000000
3,dater_race_caucasian,0.847291,0.750000,0.171429,0.039409,0.171429,0.011905
4,dater_race_other,0.769231,1.000000,0.400000,0.153846,0.400000,0.000000
5,dated_race_'Asian/Pacific Islander/Asian-Ameri...,0.876712,NaN,0.000000,0.000000,0.000000,0.000000
6,dated_race_'Black/African American',0.833333,1.000000,0.500000,0.166667,0.500000,0.000000
7,dated_race_'Latino/Hispanic American',0.826087,0.500000,0.250000,0.086957,0.250000,0.052632
8,dated_race_caucasian,0.857143,0.857143,0.166667,0.032258,0.166667,0.005525
9,dated_race_other,0.812500,0.750000,0.600000,0.250000,0.600000,0.090909


In [ ]:
metrics_df.set_index(metrics_df.columns[0], inplace=True)
metrics_df2.set_index(metrics_df2.columns[0], inplace=True)

perc_change_df = percentage_change = safe_divide((metrics_df2 - metrics_df), metrics_df) * 100
perc_change_df

,accuracy,precision,recall,positive_rate,tpr,fpr
group,,,,,,
dater_race_'Asian/Pacific Islander/Asian-American',210.023310,inf,inf,-2.564103,inf,inf
dater_race_'Black/African American',255.555556,100.000000,33.333333,-11.111111,33.333333,NaN
dater_race_'Latino/Hispanic American',291.935484,NaN,NaN,-100.000000,NaN,NaN
dater_race_caucasian,218.889386,87.500000,-5.714286,-49.014778,-5.714286,-67.261905
dater_race_other,438.461538,inf,inf,115.384615,inf,NaN
dated_race_'Asian/Pacific Islander/Asian-American',265.296804,NaN,NaN,-100.000000,NaN,-100.000000
dated_race_'Black/African American',233.333333,50.000000,25.000000,100.000000,25.000000,NaN
dated_race_'Latino/Hispanic American',346.086957,inf,inf,17.391304,inf,inf
dated_race_caucasian,216.981132,157.142857,66.666667,-47.311828,66.666667,-70.718232


## 3 - Post-Processing

In [58]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, roc_auc_score # Various model evaluation metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict # train test split and cross validation accuracy/prediction functions

results = X
y_hat = clf.predict(X)
y_hat_proba = clf.predict_proba(X)

results['predicted'] = y_hat
# Append predicted probabilities to the results DataFrame
results['predicted_prob'] = y_hat_proba[:, 1]
results['actual'] = y_test


predictions = cross_val_predict(clf, X, y, cv = 10)
confusion = confusion_matrix(y, predictions)
tn, fp, fn, tp = confusion.ravel()

In [59]:
#we see the rates for each metric of the entire df

print('True Negatives:', tn)
print('False positives:', fp)
print('False Negatives',fn)
print('True Positives',tp)

True Negatives: 794
False positives: 68
False Negatives 19
True Positives 167


In [56]:
dater_groups = ["dater_race_'Asian/Pacific Islander/Asian-American'","dater_race_'Black/African American'","dater_race_'Latino/Hispanic American'", 'dater_race_caucasian','dater_race_other']

def safe_divide(n, d):
  try:
    return n/d
  except ZeroDivisionError:
    return np.nan


def compute_metrics(info, group):
    metric_df = info.loc[info[group] == 1]
    tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
    fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
    recall = safe_divide(tp, (tp + fn))
    return recall

column_names = ['group', 'recall']
metrics_df2 = pd.DataFrame(columns=column_names)
for group in dater_groups:
    recall = compute_metrics(results, group)
    metrics_df2.loc[len(metrics_df2)] = [group, recall]

metrics_df2



,group,recall
0,dater_race_'Asian/Pacific Islander/Asian-Ameri...,0.846154
1,dater_race_'Black/African American',1.000000
2,dater_race_'Latino/Hispanic American',0.666667
3,dater_race_caucasian,0.612903
4,dater_race_other,1.000000


In [34]:
results

,is_dater_male,dater_age,dated_age,age_difference,are_same_race,same_race_importance_for_dater,same_religion_importance_for_dater,attractiveness_importance_for_dated,sincerity_importance_for_dated,intelligence_importance_for_dated,...,dater_race_caucasian,dater_race_other,dated_race_'Asian/Pacific Islander/Asian-American',dated_race_'Black/African American',dated_race_'Latino/Hispanic American',dated_race_caucasian,dated_race_other,predicted,predicted_prob,actual
0,False,21,27,6,False,2.0,4.0,35.0,20.0,20.0,...,0,0,0,0,0,1,0,0,0.458944,NaN
1,False,21,22,1,False,2.0,4.0,60.0,0.0,0.0,...,0,0,0,0,0,1,0,1,0.600056,0.0
2,False,21,23,2,False,2.0,4.0,30.0,5.0,15.0,...,0,0,0,0,0,1,0,1,0.710340,1.0
3,False,21,24,3,False,2.0,4.0,30.0,10.0,20.0,...,0,0,0,0,1,0,0,1,0.690726,NaN
4,False,21,25,4,False,2.0,4.0,50.0,0.0,30.0,...,0,0,0,0,0,1,0,0,0.332573,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,True,19,20,1,False,4.0,1.0,15.0,15.0,20.0,...,0,0,0,0,0,0,1,0,0.057243,NaN
1044,True,19,21,2,False,4.0,1.0,15.0,15.0,25.0,...,0,0,0,0,0,1,0,0,0.094866,NaN
1045,True,19,20,1,False,4.0,1.0,20.0,20.0,20.0,...,0,0,0,1,0,0,0,0,0.069090,0.0
1046,True,19,21,2,False,4.0,1.0,15.0,15.0,25.0,...,0,0,0,0,0,1,0,0,0.096780,0.0


In [61]:
import numpy as np
import pandas as pd

# Function to compute recall
def safe_divide(n, d):
    try:
        return n / d
    except ZeroDivisionError:
        return np.nan

# Function to compute metrics
def compute_metrics(info, group):
    metric_df = info.loc[info[group] == 1]
    print(metric_df)
    tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
    fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
    recall = safe_divide(tp, (tp + fn))
    return recall

# Function to compute recall using custom threshold
def compute_recall_with_threshold(info, group, threshold):
    metric_df = info.loc[info[group] == 1]

    tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted_prob'] > threshold)])
    fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted_prob'] <= threshold)])
    recall = safe_divide(tp, (tp + fn))
    return recall

# Define the list of race groups
dater_groups = ["dater_race_'Asian/Pacific Islander/Asian-American'",
                "dater_race_'Black/African American'",
                "dater_race_'Latino/Hispanic American'",
                'dater_race_caucasian',
                'dater_race_other']

# Initialize an empty DataFrame to store results
column_names = ['group', 'new_recall', 'recall_threshold']
metrics_df = pd.DataFrame(columns=column_names)

# Assuming you have your list of recall thresholds named 'recall_thresholds'
recall_thresholds = [0.5, .5, 0.3, 0.2, 0.3]

# Iterate over each race group
for group, threshold in zip(dater_groups, recall_thresholds):
    # Compute recall for the group with the custom threshold
    og_recall = compute_recall_with_threshold(results, group, .5)
    recall = compute_recall_with_threshold(results, group, threshold)
    # Add the computed recall and the corresponding threshold to the DataFrame
    metrics_df = metrics_df.append({'group': group, 'new_recall': recall, 'recall_threshold': threshold, 'original_recall': og_recall}, ignore_index=True)

# Print the DataFrame
(metrics_df) 


C:\Users\arian\AppData\Local\Temp\ipykernel_10768\3021653700.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({'group': group, 'new_recall': recall, 'recall_threshold': threshold, 'original_recall': og_recall}, ignore_index=True)
C:\Users\arian\AppData\Local\Temp\ipykernel_10768\3021653700.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({'group': group, 'new_recall': recall, 'recall_threshold': threshold, 'original_recall': og_recall}, ignore_index=True)
C:\Users\arian\AppData\Local\Temp\ipykernel_10768\3021653700.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({'group': group, 'new_recall': recall, 'recall_threshold':

,group,new_recall,recall_threshold,original_recall
0,dater_race_'Asian/Pacific Islander/Asian-Ameri...,0.846154,0.5,0.846154
1,dater_race_'Black/African American',1.000000,0.5,1.000000
2,dater_race_'Latino/Hispanic American',0.833333,0.3,0.666667
3,dater_race_caucasian,0.870968,0.2,0.612903
4,dater_race_other,1.000000,0.3,1.000000
